In [1]:
import requests
import pandas as pd
from config import tmdb_key
from pprint import pprint as pprint

In [2]:
base_params = dict(api_key=tmdb_key)
year = 2018
page = 1
listing_params = dict(primary_release_year=year,region='US', sort_by='revenue.desc', include_adult='false', language='en-US', page='1')
listing_params.update(base_params)
response = requests.get('https://api.themoviedb.org/3/discover/movie', params=listing_params)
imdb_films = response.json()['results']


In [3]:
rows = []

restart = True
page_limit = True
while restart == True:
   while page_limit == True:
      for film in imdb_films:
         film_revenue = requests.get('https://api.themoviedb.org/3/movie/{}'.format(film['id']), params=listing_params)
         film_revenue = film_revenue.json()
         try:
            row = dict(title=film['title'], revenue=film_revenue['revenue'], Year=listing_params['primary_release_year'],\
                     VoteAVG=film['vote_average'], VoteCT=film['vote_count'], Popularity=film['popularity'], Date=film['release_date'],\
                        Genre=film['genre_ids'][0])
         except:
            pass   
         rows.append(row)
      #MODIFY page == x: to increase or decrease sample size
      if page == 10: 
         page_limit = False
      else:
         page += 1
         listing_params.update({'page':page})
         response = requests.get('https://api.themoviedb.org/3/discover/movie', params=listing_params)
         imdb_films = response.json()['results']
   if year == 2023: 
      restart = False
   else:
      page_limit = True
      page = 1
      year += 1
      listing_params.update({'primary_release_year':year})
      listing_params.update({'page':page})
      response = requests.get('https://api.themoviedb.org/3/discover/movie', params=listing_params)
      imdb_films = response.json()['results']
      
df = pd.DataFrame(rows, columns=['title', 'revenue', 'Year','VoteAVG', 'VoteCT', 'Popularity', 'Date', 'Genre'])
df['Date'] = pd.to_datetime(df['Date'])

In [4]:
#Copy df
df_copy = df.copy()

In [5]:
data = pd.DataFrame(
       {
              'A': [28,12,16,35,80,99,18,10751,14,36,27,10402,9648,10749,878,10770,53,10752,37],
              'B':['Action','Adventure','Animation','Comedy','Crime','Documentary','Drama','Family','Fantasy','History',\
                     'Horror','Music','Mystery','Romance','Science', 'Fiction','TV Movie','War','Western']
       }
)
map_dict = {28:'Action', 12:'Adventure'}
map_dict = {x:data['B'].tolist()[i] for i,x in enumerate(data['A'].tolist())}
df_copy['Genre'] = df_copy['Genre'].map(map_dict)
df_copy.head()

,title,revenue,Year,VoteAVG,VoteCT,Popularity,Date,Genre
0,Avengers: Infinity War,2052415039,2018,8.3,27942,198.291,2018-04-27,Adventure
1,Black Panther,1349926083,2018,7.4,21165,77.256,2018-02-16,Action
2,Jurassic World: Fallen Kingdom,1310466296,2018,6.5,10523,61.757,2018-06-22,Action
3,Incredibles 2,1242805359,2018,7.5,12137,73.992,2018-06-15,Action
4,Aquaman,1148528393,2018,6.9,13066,60.446,2018-12-21,Action


In [6]:
df_copy.to_csv("output_data/tmdb_movies.csv", index_label="Index")

In [7]:
df_grouped = df_copy.groupby('Year')['revenue']
Mean = df_grouped.mean()
Median = df_grouped.median()
SUM = df_grouped.sum()
VAR = df_grouped.var()
STD = df_grouped.std()
SEM = df_grouped.sem()

summary_dic = {
    'Sum': SUM,
    'Mean': Mean,
    'Median': Median,
    'VAR': VAR,
    "STD": STD,
    'SEM': SEM
}

dfmovie_summary = pd.DataFrame(summary_dic)
dfmovie_summary

,Sum,Mean,Median,VAR,STD,SEM
Year,,,,,,
2018,29805757747,1.490288e+08,41071189.5,7.421928e+16,2.724322e+08,1.926386e+07
2019,29961648503,1.498082e+08,38170505.5,1.039134e+17,3.223560e+08,2.279401e+07
2020,6018720886,3.009360e+07,2088769.5,6.557609e+15,8.097906e+07,5.726084e+06
2021,14685970896,7.342985e+07,13414265.0,3.454413e+16,1.858605e+08,1.314232e+07
2022,19583301146,9.791651e+07,14789929.0,6.314694e+16,2.512905e+08,1.776892e+07
2023,18959878309,9.479939e+07,11250000.0,4.351919e+16,2.086125e+08,1.475113e+07
